<a href="https://colab.research.google.com/github/tarakantaacharya/phi2_text2sql_finetuned/blob/main/Phi2_SQL_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install accelerate bitsandbytes datasets peft transformers einops torch trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import pipeline

model_name = "microsoft/phi-2"

pipe = pipeline(
    "text-generation",
    model=model_name,
    device_map="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer

In [4]:
base_model = "microsoft/phi-2"
new_model = "phi2-text2sql"

In [6]:
import os
import shutil

# Create the .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Now copy the kaggle.json file to the .kaggle folder
shutil.copy('/content/kaggle.json', '/root/.kaggle/kaggle.json')

'/root/.kaggle/kaggle.json'

In [7]:
!kaggle datasets download -d mohammadnouralawad/spider-text-sql

Dataset URL: https://www.kaggle.com/datasets/mohammadnouralawad/spider-text-sql
License(s): CC-BY-SA-4.0


In [8]:
import zipfile
# Replace 'your_zip_file.zip' with the actual path to your zip file
with zipfile.ZipFile('/content/spider-text-sql.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_files') # Replace 'extracted_files' with desired extraction location

In [9]:
import pandas as pd
df = pd.read_csv('/content/extracted_files/spider_text_sql.csv')
df.head()

,text_query,sql_command
0,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD..."
2,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ..."
3,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i..."
4,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...


In [10]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Sample DataFrame (Replace this with your actual data)
df = df.head(700)

# Concatenate columns into a single "text" column
df["text"] = "### Text Query: " + df["text_query"] + "\n### SQL Command: " + df["sql_command"]

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Remove the auto-generated index column
hf_dataset = hf_dataset.remove_columns(["__index_level_0__"]) if "__index_level_0__" in hf_dataset.column_names else hf_dataset

# **Split dataset into Train (80%) and Validation (20%)**
split_dataset = hf_dataset.train_test_split(test_size=0.2, seed=42)

# Convert to DatasetDict (For Trainer Compatibility)
dataset_dict = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

# Print dataset sizes
print(f"Train size: {len(dataset_dict['train'])}, Validation size: {len(dataset_dict['validation'])}")

# Print sample from train set
print(dataset_dict["train"][0])

Train size: 560, Validation size: 140
{'text_query': 'Return the characters for actors, ordered by age descending.', 'sql_command': 'SELECT Character FROM actor ORDER BY age DESC', 'text': '### Text Query: Return the characters for actors, ordered by age descending.\n### SQL Command: SELECT Character FROM actor ORDER BY age DESC'}


In [11]:
# Load base model(Phi-2)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
        'fc1',
        'fc2',
    ]
)
model = get_peft_model(model, peft_config)

In [13]:
import shutil
import os

folder_path = "/content/phi2-text2sql"  # Change this to your target folder

# Remove all files and subdirectories in the folder
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)  # Delete the folder and its contents
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder does not exist: {folder_path}")

Folder does not exist: /content/phi2-text2sql


In [15]:
from transformers import EarlyStoppingCallback

training_arguments = TrainingArguments(
    output_dir="./phi2-text2sql",
    num_train_epochs=10,  # Reduce to prevent overfitting
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_steps=10,
    logging_strategy="steps",
    logging_first_step=True,
    logging_dir="./logs",
    learning_rate=5e-4,  # Increase LR for better learning
    weight_decay=0.05,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    max_grad_norm=1.0,  # Clip gradients to avoid instability
    label_names=["labels"]
)

In [16]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    peft_config=peft_config,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stops training if no improvement
)

Converting train dataset to ChatML:   0%|          | 0/560 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/560 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/560 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/560 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/140 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/140 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/140 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/140 [00:00<?, ? examples/s]

In [17]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.884800,0.849516
2,0.708900,0.704236
3,0.513800,0.655855
4,0.392400,0.651422


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Epoch,Training Loss,Validation Loss
1,0.884800,0.849516
2,0.708900,0.704236
3,0.513800,0.655855
4,0.392400,0.651422
5,0.368100,0.654516
6,0.312100,0.663466


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=420, training_loss=0.5550618035452707, metrics={'train_runtime': 1671.7053, 'train_samples_per_second': 3.35, 'train_steps_per_second': 0.419, 'total_flos': 3095759949619200.0, 'train_loss': 0.5550618035452707})

In [2]:
import os

model_dir = "./phi2-text2sql"  # Your output directory
checkpoints = [f for f in os.listdir(model_dir) if f.startswith("checkpoint")]
print("Available Checkpoints:", checkpoints)

Available Checkpoints: ['checkpoint-420', 'checkpoint-280']


In [3]:
latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))[-1]  # Sort numerically
checkpoint_path = os.path.join(model_dir, latest_checkpoint)

print("Using checkpoint:", checkpoint_path)

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the latest checkpoint
model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model.eval()  # Set to evaluation mode

Using checkpoint: ./phi2-text2sql/checkpoint-420


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2560, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(

In [7]:
import torch

def generate_sql(query, model, tokenizer, max_length=200):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(query, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

import re

def generate_sql(user_query, model, tokenizer):
    input_ids = tokenizer.encode(user_query, return_tensors="pt")
    output = model.generate(
        input_ids=input_ids,
        max_length=100,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

# Example user query
user_query = """
## INPUT
Query: Find the total revenue generated in the year 2025.

## OUTPUT
SQL query:
"""

# Generate SQL from the model (Make sure `model` and `tokenizer` are defined)
generated_sql = generate_sql(user_query, model, tokenizer)

print("Purified SQL Output:", generated_sql)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Purified SQL Output: ## INPUT
Query: Find the total revenue generated in the year 2025.

## OUTPUT
SQL query:
SELECT sum(revenue) FROM financial WHERE year = 2025
